# Add Flexibility to Calculation of Allocations

Thu 13 Sep, 2018

In [1]:
import numpy as np
import pandas as pd

## Create DataFrames with test data

In [29]:
#Note that in order fo the patronage calculations to work, this DataFrame needs to contain all members.
prev_equity_df = pd.DataFrame([
    [1, 'Alice', 0, 0, 0],
    [2, 'Bob', 125, 0, 0],
    [3, 'Candice', 125, 3000, 200],
    [4, 'Darwin', 0, 0, 0],
    [5, 'Eve', 125, 1000, 100],
    [6, 'Federico', 125, 0, 700],
    [7, 'Gabi', 125, 0, 0],
    [8, 'HAL', 125, 0, 0],
    [9, 'Irene', 125, 10000, 0],
    [10, 'Jules', 0, 0, 0]
], columns = ['member_id', 'name', 'membership', 'preferred', 'other'])
prev_equity_df['equity'] = prev_equity_df['membership'] + prev_equity_df['preferred'] + prev_equity_df['other']
prev_equity_df

,member_id,name,membership,preferred,other,equity
0,1,Alice,0,0,0,0
1,2,Bob,125,0,0,125
2,3,Candice,125,3000,200,3325
3,4,Darwin,0,0,0,0
4,5,Eve,125,1000,100,1225
5,6,Federico,125,0,700,825
6,7,Gabi,125,0,0,125
7,8,HAL,125,0,0,125
8,9,Irene,125,10000,0,10125
9,10,Jules,0,0,0,0


In [30]:
contributions_df = pd.DataFrame([
    [1, "1/3/2017", 125, 'membership'],
    [2, "1/31/2017", 400, 'other'],
    [1, "2/20/2017", 5000, 'preferred'],
    [3, "3/15/2017", -2000, 'preferred'],
    [1, "4/13/2017", 500, 'other'],
    [2, "4/13/2017", 7500, 'preferred'],
    [4, "6/24/2017", 125, 'mebership'],
    [5, "7/17/2017", 3000, "preferred"],
    [6, "8/22/2017", -700, 'other'],
    [4, "10/18/2017", 20000, 'preferred'],
    [6, "11/11/2017", 1500, 'preferred'],
    [10, "5/4/2017", 125, 'membership']
], columns = ['member_id','date','amount','type'])
contributions_df

,member_id,date,amount,type
0,1,1/3/2017,125,membership
1,2,1/31/2017,400,other
2,1,2/20/2017,5000,preferred
3,3,3/15/2017,-2000,preferred
4,1,4/13/2017,500,other
5,2,4/13/2017,7500,preferred
6,4,6/24/2017,125,mebership
7,5,7/17/2017,3000,preferred
8,6,8/22/2017,-700,other
9,4,10/18/2017,20000,preferred


## Collect core functions from dividends notebook

Change percents to proportions.

In [32]:
def fraction_year_remaining(date):
    """Computes the fraction of the year remaining from a given date.
    """
    date = pd.to_datetime(date)
    offset = pd.tseries.offsets.YearBegin()
    year = date - offset
    next_year = date + offset
    #print(offset, year, next_year)
    return (next_year - date).days / (next_year - year).days

def compute_transaction_patronage(contributions_df):
    """Compute patronage for each transaction based on amount and fraction of year remaining.
    """
    contributions_df = contributions_df.copy()
    patronage = contributions_df['amount']*contributions_df['date'].apply(fraction_year_remaining)
    contributions_df['patronage'] = patronage
    return contributions_df

def compute_new_patronage(contributions_df):
    """Compute each member's patronage from new contributions for the current year.
    """
    contributions_df = compute_transaction_patronage(contributions_df)
    return contributions_df[['member_id','patronage']].groupby(by='member_id').sum()

def compute_patronage(prev_equity_df, contributions_df):
    """Compute total patronage for each member from new contributions for the current year
        and existing equity from previous years.
    """
    patronage_df = prev_equity_df.set_index('member_id')[['name', 'equity']]
    patronage_df.rename(columns={'equity': 'old_patronage'}, inplace=True)
    
    patronage_df['new_patronage'] = compute_new_patronage(contributions_df)['patronage']
    # If there were members with no contributions this year, set their new patronage to 0 (would be NaN).
    patronage_df.fillna(0, inplace=True)
    
    patronage_df['patronage'] = patronage_df['old_patronage'] + patronage_df['new_patronage']
    patronage_df['proportionate_patronage'] = patronage_df['patronage'] / patronage_df['patronage'].sum()
    
    return patronage_df

def compute_dividends(patronage_df, profit, proportion_individual=0.5):
    """Compute each member's dividend based on patronage for the year.
    """ 
    dividend_df = patronage_df[['name', 'proportionate_patronage']].copy()
    
    #Compute individual patronage allocations
    dividend_df['dividend'] = np.round(
        dividend_df['proportionate_patronage'] * profit * proportion_individual, 2)
    
    # To account for rounding amounts to the nearest cent, we add up the individual dividends
    # to get the actual amount allocated to individual net income. Then we subtract this amount
    # from the total profit to get the collective net income.
    indiv_profit = dividend_df['dividend'].sum()
    collective_profit = profit - indiv_profit
    
    # We reserve member_id=0 for the collective account (or we could simply use names as keys)
    dividend_df.loc[0] = pd.Series({
        'name': 'CollectiveAcct',
        'proportionate_patronage': collective_profit / indiv_profit,
        'dividend': collective_profit
    })
    return dividend_df

## Auxiliary functions

In [33]:
def compute_total_patronage(patronage_df):
    """Computes the total patronage for the year, i.e. the time-averaged amount of equity.
    """
    return patronage_df['patronage'].sum()

def estimate_indiv_collective_profit(profit, proportion_individual):
    """Estimate the individual and collective net income from the total net income.
    This may be different from the actual individual and collective amounts because
    of rounding individual dividends to the nearest cent.
    """
    indiv_profit = np.round(profit * proportion_individual, 2)
    collective_profit = profit - indiv_profit
    return indiv_profit, collective_profit

In [34]:
patronage_df = compute_patronage(prev_equity_df, contributions_df)
patronage_df

,name,old_patronage,new_patronage,patronage,proportionate_patronage
member_id,,,,,
1,Alice,0,4799.657534,4799.657534,0.157671
2,Bob,125,5771.232877,5896.232877,0.193694
3,Candice,3325,-1600.000000,1725.000000,0.056667
4,Darwin,0,4175.000000,4175.000000,0.137150
5,Eve,1225,1380.821918,2605.821918,0.085602
6,Federico,825,-43.561644,781.438356,0.025671
7,Gabi,125,0.000000,125.000000,0.004106
8,HAL,125,0.000000,125.000000,0.004106
9,Irene,10125,0.000000,10125.000000,0.332610


In [35]:
patronage_df.sum()

name                       AliceBobCandiceDarwinEveFedericoGabiHALIreneJules
old_patronage                                                          15875
new_patronage                                                          14566
patronage                                                              30441
proportionate_patronage                                                    1
dtype: object

In [66]:
profit = 2182.33
#Different profit values used to test rounding behavior: 2182.33, 0.04, 0.23, 1.17, 1729.45, 100934.27, 10.75, 3.27
#Note: 0.01, 0.02, 0.03 all result in 0.00 individual profit, leading to a divide by zero error for the collective proportion.
dividend_df = compute_dividends(patronage_df, profit)
dividend_df

,name,proportionate_patronage,dividend
member_id,,,
1,Alice,0.157671,172.04
2,Bob,0.193694,211.35
3,Candice,0.056667,61.83
4,Darwin,0.137150,149.65
5,Eve,0.085602,93.41
6,Federico,0.025671,28.01
7,Gabi,0.004106,4.48
8,HAL,0.004106,4.48
9,Irene,0.332610,362.93


In [67]:
estimate_indiv_collective_profit(profit, 0.5)

(1091.1600000000001, 1091.1699999999998)

In [95]:
dividend_df.sum()

name                       AliceBobCandiceDarwinEveFedericoGabiHALIreneJu...
proportionate_patronage                                              2.00003
dividend                                                             2182.33
dtype: object

## Test calculations with numpy arrays

In [76]:
a = np.array([0.25 for x in range(3)])
a

array([ 0.25,  0.25,  0.25])

In [77]:
a[0] = 0.5
a

array([ 0.5 ,  0.25,  0.25])

In [78]:
a * 4

array([ 2.,  1.,  1.])

In [83]:
a.reshape(1,-1) #Reshapes a to a 1x3 row vector

array([[ 0.5 ,  0.25,  0.25]])

In [87]:
#.reshape(-1,1) reshapes dividend_df['dividend'].values to a 11x1 column vector.
#The broadcast product with a.reshape(1,-1) will be 11x3.
b = a.reshape(1,-1) * dividend_df['dividend'].values.reshape(-1,1)
b

array([[  86.02  ,   43.01  ,   43.01  ],
       [ 105.675 ,   52.8375,   52.8375],
       [  30.915 ,   15.4575,   15.4575],
       [  74.825 ,   37.4125,   37.4125],
       [  46.705 ,   23.3525,   23.3525],
       [  14.005 ,    7.0025,    7.0025],
       [   2.24  ,    1.12  ,    1.12  ],
       [   2.24  ,    1.12  ,    1.12  ],
       [ 181.465 ,   90.7325,   90.7325],
       [   1.485 ,    0.7425,    0.7425],
       [ 545.59  ,  272.795 ,  272.795 ]])

In [88]:
b.sum()

2182.3299999999999

In [89]:
b.sum(axis=1)

array([  172.04,   211.35,    61.83,   149.65,    93.41,    28.01,
           4.48,     4.48,   362.93,     2.97,  1091.18])

In [90]:
#The row sums should equal the dividends:
b.sum(axis=1) == dividend_df['dividend']

member_id
1     True
2     True
3     True
4     True
5     True
6     True
7     True
8     True
9     True
10    True
0     True
Name: dividend, dtype: bool

In [91]:
np.round(b, 2)

array([[  86.02,   43.01,   43.01],
       [ 105.68,   52.84,   52.84],
       [  30.92,   15.46,   15.46],
       [  74.82,   37.41,   37.41],
       [  46.7 ,   23.35,   23.35],
       [  14.  ,    7.  ,    7.  ],
       [   2.24,    1.12,    1.12],
       [   2.24,    1.12,    1.12],
       [ 181.46,   90.73,   90.73],
       [   1.48,    0.74,    0.74],
       [ 545.59,  272.79,  272.79]])

In [92]:
np.round(b, 2).sum()

2182.29

In [93]:
np.round(b, 2).sum(axis=1)

array([  172.04,   211.36,    61.84,   149.64,    93.4 ,    28.  ,
           4.48,     4.48,   362.92,     2.96,  1091.17])

In [94]:
#After rounding, the row sums no longer equal the dividends:
np.round(b, 2).sum(axis=1) == dividend_df['dividend']

member_id
1      True
2     False
3     False
4     False
5     False
6     False
7      True
8      True
9     False
10    False
0     False
Name: dividend, dtype: bool

In [97]:
np.array([1,2,3] if False else [5,6,7,8]) #Woo hoo, inline if statements!

array([5, 6, 7, 8])

In [99]:
b

array([[  86.02  ,   43.01  ,   43.01  ],
       [ 105.675 ,   52.8375,   52.8375],
       [  30.915 ,   15.4575,   15.4575],
       [  74.825 ,   37.4125,   37.4125],
       [  46.705 ,   23.3525,   23.3525],
       [  14.005 ,    7.0025,    7.0025],
       [   2.24  ,    1.12  ,    1.12  ],
       [   2.24  ,    1.12  ,    1.12  ],
       [ 181.465 ,   90.7325,   90.7325],
       [   1.485 ,    0.7425,    0.7425],
       [ 545.59  ,  272.795 ,  272.795 ]])

In [100]:
b.round(2)

array([[  86.02,   43.01,   43.01],
       [ 105.68,   52.84,   52.84],
       [  30.92,   15.46,   15.46],
       [  74.82,   37.41,   37.41],
       [  46.7 ,   23.35,   23.35],
       [  14.  ,    7.  ,    7.  ],
       [   2.24,    1.12,    1.12],
       [   2.24,    1.12,    1.12],
       [ 181.46,   90.73,   90.73],
       [   1.48,    0.74,    0.74],
       [ 545.59,  272.79,  272.79]])

In [101]:
b

array([[  86.02  ,   43.01  ,   43.01  ],
       [ 105.675 ,   52.8375,   52.8375],
       [  30.915 ,   15.4575,   15.4575],
       [  74.825 ,   37.4125,   37.4125],
       [  46.705 ,   23.3525,   23.3525],
       [  14.005 ,    7.0025,    7.0025],
       [   2.24  ,    1.12  ,    1.12  ],
       [   2.24  ,    1.12  ,    1.12  ],
       [ 181.465 ,   90.7325,   90.7325],
       [   1.485 ,    0.7425,    0.7425],
       [ 545.59  ,  272.795 ,  272.795 ]])

In [106]:
dividend_df['dividend'] - b.round(2)[:,1:].sum(axis=1)

member_id
1      86.02
2     105.67
3      30.91
4      74.83
5      46.71
6      14.01
7       2.24
8       2.24
9     181.47
10      1.49
0     545.60
Name: dividend, dtype: float64

In [113]:
#Make first year payments irregular
c = b.round(2)
c[:,0] = np.round(dividend_df['dividend'] - c[:,1:].sum(axis=1),2)
c

array([[  86.02,   43.01,   43.01],
       [ 105.67,   52.84,   52.84],
       [  30.91,   15.46,   15.46],
       [  74.83,   37.41,   37.41],
       [  46.71,   23.35,   23.35],
       [  14.01,    7.  ,    7.  ],
       [   2.24,    1.12,    1.12],
       [   2.24,    1.12,    1.12],
       [ 181.47,   90.73,   90.73],
       [   1.49,    0.74,    0.74],
       [ 545.6 ,  272.79,  272.79]])

In [109]:
c.sum()

2182.3299999999999

In [112]:
#Make last year payments irregular
d = b.round(2)
d[:,-1] = dividend_df['dividend'] - d[:,:-1].sum(axis=1)
d

array([[  86.02,   43.01,   43.01],
       [ 105.68,   52.84,   52.83],
       [  30.92,   15.46,   15.45],
       [  74.82,   37.41,   37.42],
       [  46.7 ,   23.35,   23.36],
       [  14.  ,    7.  ,    7.01],
       [   2.24,    1.12,    1.12],
       [   2.24,    1.12,    1.12],
       [ 181.46,   90.73,   90.74],
       [   1.48,    0.74,    0.75],
       [ 545.59,  272.79,  272.8 ]])

In [121]:
len(d)

11

## New function for calculating yearly allocations

In [136]:
def compute_allocations(dividend_df,
                        year,
                        irregular_payment='last',
                        first_year_proportion=0.5,
                        n_years=3,
                        pos_distribution = None,
                        neg_distribution = None):
    """Computes allocations over next n_years years after the dividend year.
    """
    
    if pos_distribution is None:
        #Default for a positive dividend is to evenly evenly divide what's left over after
        #the first year over the remaining over n-1 years.
        if n_years > 1:
            pos_distribution = [(1-first_year_proportion) / (n_years-1) for _ in range(n_years)]
            pos_distribution[0] = first_year_proportion
        else:
            pos_distribution = [1]
            
    if neg_distribution is None:
        #Default for a negative dividend is to evenly divide it over all n years.
        neg_distribution = [1.0 / n_years for _ in range(n_years)]
    
    #Check whether the profit is positive or negative, and use the corresponding distribution;
    #convert the distribution from a list to a numpy array to perform math with it.
    profit = dividend_df['dividend'].sum()
    distribution = np.array(pos_distribution if profit >= 0 else neg_distribution)
    
    #In case the distribution was explicitly passed, make sure n_years matches the actual length.
    n_years = len(distribution)
    
    #Create a new DataFrame for the allocations by dropping the collective account and
    #the proportionate patronage column from dividend_df.
    allocation_df = dividend_df.drop(index=0, columns='proportionate_patronage')
    
    #Use broadcasting to compute all dividends with one multiplication, and round them to nearest cent.
    allocations = (distribution.reshape(1,-1) * allocation_df['dividend'].values.reshape(-1,1)).round(2)
    
    #Adjust the first or last payment to account for rounding, by subtracting the sum of the remaining
    #payments from the member's actual dividend.
    if irregular_payment == 'first':
        allocations[:,0] = np.round(allocation_df['dividend'] - allocations[:,1:].sum(axis=1), 2)
    elif irregular_payment == 'last':
        allocations[:,-1] = np.round(allocation_df['dividend'] - allocations[:,:-1].sum(axis=1), 2)
    else: #We could actually allow passing a specific index if we wanted to complicate things...
        raise ValueError("irregular_payment must be 'first' or 'last'.")
        
    #Create new column labels for the years year+1, year+2,...,year+n
    new_columns = [str(y) for y in range(year+1, year+n_years+1)]
    
    #Concatenate the existing allocation dataframes horizontally with the computed allocations.
    allocation_df = pd.concat([allocation_df,
                              pd.DataFrame(allocations, index=allocation_df.index, columns = new_columns)],
                             axis=1)
    
    #Rename the 'dividend' column.
    allocation_df.rename(columns={'dividend': str(year)+'_dividend'}, inplace=True)
    
    return allocation_df

In [147]:
allocation_df = compute_allocations(dividend_df, 2017, first_year_proportion=0.7, n_years=3)
allocation_df

,name,2017_dividend,2018,2019,2020
member_id,,,,,
1,Alice,172.04,120.43,25.81,25.80
2,Bob,211.35,147.94,31.70,31.71
3,Candice,61.83,43.28,9.27,9.28
4,Darwin,149.65,104.76,22.45,22.44
5,Eve,93.41,65.39,14.01,14.01
6,Federico,28.01,19.61,4.20,4.20
7,Gabi,4.48,3.14,0.67,0.67
8,HAL,4.48,3.14,0.67,0.67
9,Irene,362.93,254.05,54.44,54.44


In [148]:
allocation_df.sum()

name             AliceBobCandiceDarwinEveFedericoGabiHALIreneJules
2017_dividend                                              1091.15
2018                                                        763.82
2019                                                        163.67
2020                                                        163.66
dtype: object

In [149]:
allocation_df.loc[:,'2018':].sum(axis=1)

member_id
1     172.04
2     211.35
3      61.83
4     149.65
5      93.41
6      28.01
7       4.48
8       4.48
9     362.93
10      2.97
dtype: float64